In [12]:
from itertools import chain
from pathlib import Path
from shutil import rmtree

import numpy as np
import torch
from nvidia.dali import fn, pipeline_def
from torch.utils.data import Dataset
from torchvision.io.image import ImageReadMode, read_image, write_png
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torchvision.transforms.functional import InterpolationMode, resize

In [28]:
class OcelotDataset(Dataset):
    def __init__(self, directory_path: str):
        self.directory = Path(directory_path)
        self._initialize_dataset()

    def _initialize_dataset(self):
        image_length = len(list(self.directory.glob("images/train/tissue/*.jpg")))
        label_length = len(list(self.directory.glob("annotations/train/tissue/*.png")))
        uncertainty_length = len(list(self.directory.glob("*_uncertainty.bin")))

        if image_length != label_length != uncertainty_length:
            raise Exception(
                f"Image length {image_length} != label length {label_length} != uncertainty_length {uncertainty_length}"
            )

        self.dataset_length = image_length

    def __len__(self):
        return self.dataset_length

    def get_mask(self, label):
        return (
            torch.concatenate(
                [
                    (label == 1),  # BACKGROUND class
                    (label == 2),  # CANCER class
                    (label == 255),  # UNKNOWN class
                ]
            )
            * 255
        )

    def __getitem__(self, index):
        filled_index = str(index + 1).zfill(3)
        image_path = str(
            self.directory.joinpath(f"images/train/tissue/{filled_index}.jpg")
        )
        label_path = str(
            self.directory.joinpath(f"annotations/train/tissue/{filled_index}.png")
        )
        image = read_image(image_path, ImageReadMode.RGB)
        label = read_image(label_path, ImageReadMode.GRAY)
        label = self.get_mask(label).to(torch.uint8)
        return image, label

In [ ]:
directory = "/mnt/storage/ocelot2023_v1.0.1/images/train/tissue/"


@pipeline_def(batch_size=8, num_threads=4, device_id=0)
def pipe():
    image_files, label_files = fn.external_source()
    images = fn.decoders.image(image_files, device="cpu")

    return images


pipe = pipe()
pipe.build()
data = pipe.run()
print(data)

RuntimeError: Critical error when building pipeline:
Error in CPU operator `nvidia.dali.fn.readers.file`,
which was used in the pipeline definition with the following traceback:

  File "/tmp/ipykernel_5629/2598378851.py", line 6, in pipe
    image_files, label_files = fn.readers.file(file_root=directory)

encountered:

Assert on "SizeImpl() > 0" failed: No files found.
C++ context: [/opt/dali/dali/operators/reader/loader/file_label_loader.h:192] 
Current pipeline object is no longer valid.

In [ ]:
image, label = dataset[0]

plt.figure(figsize=(9, 16))
plt.subplot(2, 1, 1)
plt.imshow(image.permute([1, 2, 0]))
plt.subplot(2, 1, 2)
plt.imshow(label.permute([1, 2, 0]))

In [2]:
random_tensor = torch.rand([128, 8, 1024, 1024], device="cuda")
np.save("../data.npy", random_tensor.cpu())